### Sale Prediciton by Time Analysis

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import catboost as cb
from tqdm import tqdm
from holidays import CountryHoliday



In [2]:
test = pd.read_csv(r'C:\Users\logan\Desktop\Sale prediction using time analysis\test.csv')
train = pd.read_csv(r'C:\Users\logan\Desktop\Sale prediction using time analysis\train.csv')

## Things needed in this project:
- Holidays
- GPD per country
- Accountig for covid (probs with smoothing)
- Split per seasons/quarters/etc.\
- Catboost optuna
---
Also, target feature is right skewed (maybe log transform?)

In [3]:
# First change the date from string to date and set it to index and create extra features

# Log transform for target
train["num_sold"] = np.log(train["num_sold"])

def extractDate(df):
    df.date = pd.to_datetime(df.date)
    df['year'] = df.date.dt.year
    df['month'] = df.date.dt.month
    df['dayofmonth'] = df.date.dt.day
    df['dayofweek'] = df.date.dt.dayofweek
    df['dayname'] = df.date.dt.strftime('%A')
    df['dayofyear'] = df.date.dt.dayofyear
    df['quarter'] = df.date.dt.quarter
    
extractDate(train)
extractDate(test)

In [4]:
print(train['country'].unique())
print(test['country'].unique())
print('-'*100)
print(train['year'].unique())
print(test['year'].unique())

['Argentina' 'Canada' 'Estonia' 'Japan' 'Spain']
['Argentina' 'Canada' 'Estonia' 'Japan' 'Spain']
----------------------------------------------------------------------------------------------------
[2017 2018 2019 2020 2021]
[2022]


In [5]:
# Creating features to denote holiday status of days taking country into account
# This function was not written by me https://www.kaggle.com/code/yeoyunsianggeremie/s3e19-catboost-smoothing-post-processing#Predict-2022-result
def GetAndAddHolidays(df):
    # Get country-specific holidays
    country_list = ['Argentina', 'Canada', 'Estonia', 'Japan', 'Spain']
    years = np.arange(df['year'].min(), df['year'].max() + 1)
    holidays = pd.DataFrame(columns=['date', 'holiday', 'country'])

    for country in tqdm(country_list):
        for h in CountryHoliday(country, years=years).items():
            i=len(holidays)
            holidays.loc[i,'date']=pd.to_datetime(h[0])
            holidays.loc[i,'holiday']=h[1]
            holidays.loc[i,'country']=country

    # Add common holidays from Dec 24 to Dec 31 for each country
    common_holidays = pd.DataFrame({
    'date': pd.to_datetime(['2017-04-16','2017-12-24', '2017-12-25', '2017-12-26', '2017-12-27', '2017-12-28', '2017-12-29', '2017-12-30', '2017-12-31',
                            '2018-04-01','2018-12-24', '2018-12-25', '2018-12-26', '2018-12-27', '2018-12-28', '2018-12-29', '2018-12-30', '2018-12-31',
                            '2019-04-21','2019-12-24', '2019-12-25', '2019-12-26', '2019-12-27', '2019-12-28', '2019-12-29', '2019-12-30', '2019-12-31',
                            '2020-04-12','2020-12-24', '2020-12-25', '2020-12-26', '2020-12-27', '2020-12-28', '2020-12-29', '2020-12-30', '2020-12-31',
                            '2021-04-04','2021-12-24', '2021-12-25', '2021-12-26', '2021-12-27', '2021-12-28', '2021-12-29', '2021-12-30', '2021-12-31',
                            '2022-04-17','2022-12-24', '2022-12-25', '2022-12-26', '2022-12-27', '2022-12-28', '2022-12-29', '2022-12-30', '2022-12-31']),
    'holiday': 'Holiday Season'
    })

    country_common_holidays = pd.DataFrame()
    for country in country_list:
        temp_common_holidays = common_holidays.copy()
        temp_common_holidays['country'] = country
        country_common_holidays = pd.concat([country_common_holidays, temp_common_holidays], ignore_index=True)

    holidays = pd.concat([holidays, country_common_holidays], ignore_index=True)
    holidays['date'] = pd.to_datetime(holidays['date'])
    holidays['isHoliday']=1
    
    
    # Merge holidays with the original DataFrame based on date and country
    df = pd.merge(df, holidays, how='left', on=['date', 'country'])
    df['holiday'] = df['holiday'].fillna('Not Holiday')
    df['isHoliday'] = df['isHoliday'].fillna(0)
    
    # Add Weekend columns
    #df['Friday'] = (df['dayofweek'] == 4).astype(int)
    #df['Saturday'] = (df['dayofweek'] == 5).astype(int)
    #df['Sunday'] = (df['dayofweek'] == 6).astype(int)
        
    # Remove duplicates from df
    df = df.drop_duplicates(subset=['date', 'country','product','store'], keep='first').reset_index().drop(columns = ["index"])

    return df, holidays


In [6]:
train,TrainHolidays = GetAndAddHolidays(train)
test,TestHolidays = GetAndAddHolidays(test)

100%|██████████| 5/5 [00:00<00:00, 75.76it/s]


In [7]:
# Extra transformations common for time series aka cyclical enconding
def seasonality_features(df):
    df['month_sin'] = np.sin(2*np.pi*df.month/12)
    df['month_cos'] = np.cos(2*np.pi*df.month/12)
    df['day_sin'] = np.sin(2*np.pi*df.dayofmonth/24)
    df['day_cos'] = np.cos(2*np.pi*df.dayofmonth/24)
    return df

train = seasonality_features(train)
test = seasonality_features(test)

In [8]:
train.head()

,id,date,country,store,product,num_sold,year,month,dayofmonth,dayofweek,dayname,dayofyear,quarter,holiday,isHoliday,month_sin,month_cos,day_sin,day_cos
0,0,2017-01-01,Argentina,Kaggle Learn,Using LLMs to Improve Your Coding,4.143135,2017,1,1,6,Sunday,1,1,Año Nuevo,1.0,0.5,0.866025,0.258819,0.965926
1,1,2017-01-01,Argentina,Kaggle Learn,Using LLMs to Train More LLMs,4.189655,2017,1,1,6,Sunday,1,1,Año Nuevo,1.0,0.5,0.866025,0.258819,0.965926
2,2,2017-01-01,Argentina,Kaggle Learn,Using LLMs to Win Friends and Influence People,2.197225,2017,1,1,6,Sunday,1,1,Año Nuevo,1.0,0.5,0.866025,0.258819,0.965926
3,3,2017-01-01,Argentina,Kaggle Learn,Using LLMs to Win More Kaggle Competitions,4.077537,2017,1,1,6,Sunday,1,1,Año Nuevo,1.0,0.5,0.866025,0.258819,0.965926
4,4,2017-01-01,Argentina,Kaggle Learn,Using LLMs to Write Better,3.891820,2017,1,1,6,Sunday,1,1,Año Nuevo,1.0,0.5,0.866025,0.258819,0.965926


In [9]:
# Adding GDP info per country and year

gdp = {
    'year': [2017, 2018, 2019, 2020, 2021, 2022],
    'Argentina': [14613, 11795, 9963, 8496, 10636, 13686],
    'Canada': [45129, 46547, 46374, 43349, 52358, 54966],
    'Estonia': [20437, 23165, 23424, 23595, 27943, 28332],
    'Japan': [38834, 39751, 40416, 39986, 39827, 33815],
    'Spain': [28185, 30379, 29581, 26959, 30103, 29350]
}

# Create a DataFrame from the dictionary
gdp_df = pd.DataFrame(gdp)

# Step 3: Melt the DataFrame to reshape it
gdp_df = pd.melt(gdp_df, id_vars=['year'], var_name='country', value_name='gdp')

# Step 5: Sort the DataFrame by 'country' and 'year'
gdp_df = gdp_df.sort_values(by=['country', 'year']).reset_index(drop=True)

gdp_df.head(10)

,year,country,gdp
0,2017,Argentina,14613
1,2018,Argentina,11795
2,2019,Argentina,9963
3,2020,Argentina,8496
4,2021,Argentina,10636
5,2022,Argentina,13686
6,2017,Canada,45129
7,2018,Canada,46547
8,2019,Canada,46374
9,2020,Canada,43349


In [10]:
def add_gdp_feature(df_main, df_gdp):
    # Step 1: Merge the DataFrames on 'country' and 'year'
    merged_df = pd.merge(df_main, df_gdp, on=['country', 'year'], how='left')

    # Step 2: Add the 'gdp' values from the second DataFrame to the first DataFrame
    df_main['gdp'] = merged_df['gdp']

    # Step 3: Handle missing GDP values (if any)
    # For example, you can fill missing GDP values with 0
    df_main['gdp'].fillna(0, inplace=True)

    # Alternatively, you can drop rows with missing GDP values
    # df_main.dropna(subset=['gdp'], inplace=True)

    # Step 4: Return the updated DataFrame
    return df_main

train_full = add_gdp_feature(train, gdp_df)
test_full = add_gdp_feature(test, gdp_df)

In [11]:
# Missing dropping id and set date as index then train with optuna
train_ready = train_full.drop('id', axis=1)
train_ready.set_index('date', inplace=True)

test_ready = test_full.drop('id', axis=1)
test_ready.set_index('date', inplace=True)



In [12]:
# Defining categorical features and model stuff
categorical = ['country', 'store', 'product', 'dayname', 'holiday']

# Cat params
params = {
    
    'n_estimators': 733, 
    'learning_rate': 0.1252297003236841, 
    'depth': 9, 
    'l2_leaf_reg': 3.783310969240252, 
    'subsample': 0.5758950122641308, 
    'colsample_bylevel': 0.1360009677406299,
    'random_state': 42,
    'verbose': False
    
}

model = cb.CatBoostRegressor(**params, cat_features = categorical)

In [13]:
train_ready.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 136950 entries, 2017-01-01 to 2021-12-31
Data columns (total 18 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   country     136950 non-null  object 
 1   store       136950 non-null  object 
 2   product     136950 non-null  object 
 3   num_sold    136950 non-null  float64
 4   year        136950 non-null  int64  
 5   month       136950 non-null  int64  
 6   dayofmonth  136950 non-null  int64  
 7   dayofweek   136950 non-null  int64  
 8   dayname     136950 non-null  object 
 9   dayofyear   136950 non-null  int64  
 10  quarter     136950 non-null  int64  
 11  holiday     136950 non-null  object 
 12  isHoliday   136950 non-null  float64
 13  month_sin   136950 non-null  float64
 14  month_cos   136950 non-null  float64
 15  day_sin     136950 non-null  float64
 16  day_cos     136950 non-null  float64
 17  gdp         136950 non-null  int64  
dtypes: float64(6), int64(7), obj

In [15]:
# Split and testing with 2021
X_train = train_ready[train_ready.index < "2021-01-01"].drop(columns = ["num_sold"])
y_train = train_ready[train_ready.index < "2021-01-01"]["num_sold"].copy()
X_val = train_ready[train_ready.index >= "2021-01-01"].drop(columns = ["num_sold"])
y_val = train_ready[train_ready.index >= "2021-01-01"]["num_sold"].copy()

In [16]:
# Fitting
model.fit(X_train, y_train, eval_set=(X_val, y_val), silent=True)

# SMAPE stuff
def smape(A, F):
    return 100/len(A) * np.sum(2 * np.abs(F - A) / (np.abs(A) + np.abs(F)))

training_predictions = np.exp(model.predict(X_val))

In [17]:
print("SMAPE score Jan to Mar 2021:",smape(np.exp(y_val)[:6750],np.round(training_predictions[:6750])))
print("SMAPE score Apr to Dec 2021:",smape(np.exp(y_val)[6750:],np.round(training_predictions[6750:])))

SMAPE score Jan to Mar 2021: 17.01929144497545
SMAPE score Apr to Dec 2021: 17.21414524376662


In [19]:
# Train with full and predict
X = train_ready.drop(columns = ["num_sold"])
y = train_ready["num_sold"].copy()

model.fit(X, y, silent=True)

# Prediction transformed back from log
y_pred = np.exp(model.predict(test_ready))

submission = pd.DataFrame()
submission['id'] = test['id']
submission['num_sold'] = y_pred

submission.to_csv('submission_final.csv', index=False)